In [1]:
!date

Fri Feb 28 16:24:59 PST 2020


# 10x correlation on unique isoforms

In [1]:
import anndata
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as scanp
from scipy.stats import ks_2samp, ttest_ind
from scipy import stats
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from umap import UMAP
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
import ast
import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [2]:
path = "/home/sina/projects/mop/mop_10xv3"
file_paths = glob.glob(path + "/tcc_gene_out*/counts_filtered/adata.h5ad")

In [3]:
ss = anndata.read_h5ad("../cell_ranger_annotation/isoform.h5ad")
ss_isoforms = ss.var.transcript_id.values

In [4]:
file_paths = np.sort(file_paths).tolist()

In [5]:
file_paths

['/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_A01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_B01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_C01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_D01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_E01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_F01_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_G12_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_181211_01_H12_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_out_L8TX_190430_01_A08_S01_L003/counts_filtered/adata.h5ad',
 '/home/sina/projects/mop/mop_10xv3/tcc_gene_o

In [6]:
batch_id = [
"3L8TX_181211_01_A01",
"4L8TX_181211_01_B01",
"5L8TX_181211_01_C01",
"6L8TX_181211_01_D01",
"7L8TX_181211_01_E01",
"8L8TX_181211_01_F01",
"1L8TX_181211_01_G12",
"2L8TX_181211_01_H12",
"9L8TX_190430_01_A08",    
"10L8TX_190430_01_B08",
"11L8TX_190430_01_F08",    
"12L8TX_190430_01_G08"]

In [ ]:
# for each file, find out its non_unique_ecs, append to list, then load it with only the unique ones

In [7]:
%%time
nu_ecs = []
files = []
for fidx, f in enumerate(file_paths):
    print("{} of {}".format(fidx+1, len(file_paths)))
    
    adata = anndata.read_h5ad(f)
    
    adata.var["transcript_ids"] = adata.var["transcript_ids"].apply(lambda x: ast.literal_eval(x))
    adata.var["ec_len"] = adata.var["transcript_ids"].apply(lambda x: len(x))
    tmp_nu_ecs = adata[:,adata.var["ec_len"]>1].var["transcript_ids"].explode().unique()
    
    nu_ecs.append(tmp_nu_ecs)
    
    adata = adata[:,adata.var.ec_len==1]
    adata.var["transcript_id"] = adata.var.transcript_ids.apply(lambda x: x[0])
    adata.var["transcript_id_nov"] = adata.var["transcript_id"].str.slice(0,18)
    
    adata = adata[:,adata.var["transcript_id_nov"].isin(ss_isoforms)]
    
    files.append(adata)

1 of 12


Trying to set attribute `.var` of view, copying.


2 of 12


Trying to set attribute `.var` of view, copying.


3 of 12


...

11 of 12


Trying to set attribute `.var` of view, copying.


12 of 12


Trying to set attribute `.var` of view, copying.


CPU times: user 24min 32s, sys: 1min 8s, total: 25min 41s
Wall time: 26min 11s


In [8]:
%%time
adata = files[0].concatenate(*files[1:], batch_categories=batch_id)

CPU times: user 15 s, sys: 4.87 s, total: 19.9 s
Wall time: 19.9 s


In [9]:
adata

AnnData object with n_obs × n_vars = 169200 × 84364 
    obs: 'batch'
    var: 'transcript_ids', 'ec_len', 'transcript_id', 'transcript_id_nov'

In [11]:
ss

AnnData object with n_obs × n_vars = 6043 × 84364 
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_TPM', 'leiden', 'louvain'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'gene_TPM', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'louvain', 'neighbors'
    obsm: 'X_nca', 'X_nca_tsne', 'X_pca', 'X_pca_tsne'
    layers: 'X', 'log1p'

In [4]:
#adata.write("../cell_ranger_annotation/10xv3_unique_isoforms.h5ad")

In [5]:
adata = anndata.read_h5ad("../cell_ranger_annotation/10xv3_unique_isoforms.h5ad")

# Normalize and scale data

In [6]:
adata.layers["X"] = adata.X

In [7]:
from sklearn.preprocessing import normalize

In [8]:
mtx = normalize(adata.X, norm='l1', axis=1)

In [10]:
scale = 1000000
adata.X = mtx*scale
adata.layers["log1p"] = np.log1p(mtx*scale)

In [11]:
scanp.pp.log1p(adata)

## Highly variable genes

In [13]:
adata

AnnData object with n_obs × n_vars = 169200 × 84364 
    obs: 'batch'
    var: 'transcript_ids', 'ec_len', 'transcript_id', 'transcript_id_nov'
    uns: 'log1p'
    layers: 'X', 'log1p'

In [14]:
num_TSNE = 2
state = 42
metric = "euclidean"
n_neighbors = 30
num_PCA = 50
num_NCA = 10

# Filtering criteria
cell_threshold = 0.5e6
disp_threshold = 10

mito_criteria = 50

n_top_genes = 5000

n_bins = 20

flavor="seurat"

scale_clip = 10

In [15]:
scanp.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor=flavor, n_bins=n_bins)
hvg_mask = adata.var.highly_variable.values

In [16]:
scanp.pp.scale(adata, max_value=scale_clip)

MemoryError: Unable to allocate 106. GiB for an array with shape (169200, 84364) and data type float64

## Remove non unique isoforms

In [14]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [24]:
tmp = flatten(nu_ecs)
nu_ecs_list = np.unique(tmp)

In [29]:
nu_ecs_list.shape

(109408,)

In [ ]:
unique = adata.var.transcript_id_nov.values

In [ ]:
u = np.setdiff1d(np.sort(unique),np.sort(nu_ecs))